In [3]:
import os, sys
HOME = os.environ['HOME']  # change if necessary
sys.path.append(f'{HOME}/Finite-groups/src')

In [4]:
import torch as t
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
from jaxtyping import Float
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
import pandas as pd
from typing import Union
from einops import repeat
from huggingface_hub import snapshot_download
from huggingface_hub.utils import disable_progress_bars


from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from model_utils import *
from group_utils import *
%load_ext autoreload
%autoreload 2

Syntax warning: Unbound global variable in /usr/share/gap/pkg/browse/PackageIn\
fo.g:73
  if not IsKernelExtensionAvailable("Browse", "ncurses") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/cddinterface/Pac\
kageInfo.g:85
    if not IsKernelExtensionAvailable("CddInterface") then
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/crypting/Package\
Info.g:82
   if not IsKernelExtensionAvailable("crypting") then
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/curlinterface/Pa\
ckageInfo.g:114
  if not IsKernelExtensionAvailable("curlinterface", "curl") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/cvec/PackageInfo\
.g:114
  if not IsKernelExtensionAvailable("cvec") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/datastruct

In [7]:

device = t.device("cuda" if t.cuda.is_available() else "cpu")
MODEL_DIR = '2024-12-20_19-12-16_S4_A4x2_MLP2_32_ubias_wd2e-5_BIG'
disable_progress_bars()
local_dir = f'{HOME}/models/{MODEL_DIR}'
if not os.path.exists(local_dir):
    snapshot_download(repo_id=f'wiwu2390/{MODEL_DIR}', local_dir=local_dir)
models, params = load_models(local_dir, sel='final')
models = models[0]
data = GroupData(params)
group = data.groups[0]

Intersection size: 336/576 (0.58)
Added 336 elements from intersection
Added 0 elements from group 0: S(4)
Added 0 elements from group 1: times(A(4), Z(2))
Train set size: 336/576 (0.58)


/home/wilson/Finite-groups/src/model_utils.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(t.load(model_path, map_location=device))


In [35]:
loss_dict = test_loss(models.to(device), data)
g0_grokked = (loss_dict['G0_acc'] > 0.999)
g1_grokked = (loss_dict['G1_acc'] > 0.999)
print('G0 grokked', g0_grokked.sum())
print('G1 grokked', g1_grokked.sum())
grokked_idx = (g0_grokked | g1_grokked).nonzero().flatten()
labels = g0_grokked[grokked_idx] 
len(grokked_idx)

G0 grokked tensor(603)
G1 grokked tensor(43)


646

In [34]:
grokked_models = models[grokked_idx]
len(grokked_models)

646

In [27]:
grokked_params = []
for inst in range(len(grokked_models)):
    cur_params = t.concatenate([p.data.flatten() for p in grokked_models[inst].parameters()], dim=0)
    grokked_params.append(cur_params)
grokked_params = t.stack(grokked_params, dim=0)
grokked_params.shape

torch.Size([646, 4376])

In [29]:
N = models.N
test_inputs = t.tensor(list(product(range(N), repeat=2)), device=device)
grokked_outputs = grokked_models(test_inputs)
grokked_outputs = einops.rearrange(grokked_outputs, 'batch instance vocab -> instance (batch vocab)')
grokked_outputs.shape

torch.Size([646, 13824])

In [40]:
train_size = int(0.8 * grokked_outputs.shape[0])
train_params = grokked_params[:train_size].cpu().numpy()
test_params = grokked_params[train_size:].cpu().numpy()
train_outputs = grokked_outputs[:train_size].detach().cpu().numpy()
test_outputs = grokked_outputs[train_size:].detach().cpu().numpy()
train_labels = labels[:train_size].cpu().numpy()
test_labels = labels[train_size:].cpu().numpy()

In [74]:
from sklearn import svm
param_clf = svm.SVC(kernel='linear', class_weight='balanced', C=100)
param_clf.fit(train_params, train_labels)

SVC(C=100, class_weight='balanced', kernel='linear')

In [75]:
acc = lambda clf, X, y: ((clf.predict(X) == y)[y.nonzero()].mean(), (clf.predict(X) == y)[(~y).nonzero()].mean())

In [76]:
acc(param_clf, train_params, train_labels), acc(param_clf, test_params, test_labels)

((np.float64(1.0), np.float64(1.0)), (np.float64(1.0), np.float64(0.0)))

In [77]:
output_clf = svm.SVC(kernel='linear', class_weight='balanced')
output_clf.fit(train_outputs, train_labels)

SVC(class_weight='balanced', kernel='linear')

In [78]:
acc(output_clf, train_outputs, train_labels), acc(output_clf, test_outputs, test_labels)

((np.float64(1.0), np.float64(1.0)), (np.float64(1.0), np.float64(1.0)))